# Preprocesado de datos
Hata ahora hemos trabajado con conjuntos de datos ordenados (*tidy data*) pero no suele ser lo habitual. Antes de emplear cualquier tipo de análisis estadístico de los datos, es necesario que se cumpla que:
- Cada variable está en una columna
- Cada registro está en una única fila
- Todos los valores están en una única celda

Es decir, necesitamos *Tidy Data* según Hadley Wickham (https://vita.had.co.nz/papers/tidy-data.pdf)

El objetivo de esta práctica es aprender a unir varios conjuntos de datos y realizar un reconocimiento de los mismos en busca de valores nulos (*NA*) y valores atípicos (outliers) y ver cómo solucionar distintos tipos de problemas asociados a la estructura del conjunto de datos.

Para realizar la práctica, vamos a importar datos sobre el número de suicidios y el grado de desorden mental por  países en un periodo de tiempo. Para ello, se han tomado datos de 3 variables:
 - “suicide_death_rates”
 - “mental_health_disorders”
 - “population” 
 
por países entre los años 1990 y 2017 de la web https://ourworldindata.org.

Vamos a comenzar juntando los conjuntos de datos **Suicide death rates** y **Mental health disorders** teniendo en cuenta que la variable `mental_health_disorders` no tiene datos del año 2017.
En base a las variables `country_name`, `country_code` y `year` juntaremos los 2 conjuntos de datos en un dataframe llamado **joined_rates_data**. Posteriormente, añadiremos (a la izquierda) los datos de población basados en las variables comunes `country_name`, `country_code` y `year` como antes.
El dataframe final **joined_rates_data** va a contener las variables 
- `country_name`: nombre del país
- `country_code`: acrónimo del nombre del país
- `year`: año del dato
- `suicide_rate` : muertes por suicidio por cada 100000 habitantes 
- `mental_disorder_count`: número de personas con desórdenes mentales 
- `population_count`: número de habitantes de un país un año determinado

Para una mejor comprensión de los datos, calcularemos los valores medios de las variables `suicide_death_rates` y `Mental disorder rate` así como la proporción de suicidios por desorden mental.

Para visualizar las distribuciones de ratio de suicidios y porcentajes, representaremos histogramas encontrando asimetría a la derecha. Haremos una transformación de las variables mediante transformación de raíz cuadrada.

Finalmente, realizaremos un contraste de hipótesis (t-test) para estudiar la relación entre proporción de suicidios y desórdenes mentales y concluiremos que, efectivamente, hay una relación estadísticamente significativa (confianza del 95%) de que están relacionadas. 

In [ ]:
# Librerías
# Si alguna no está instalada en su equipo use este código
# install.packages("nombre de la librería")
library(readr) # importar datos
library(dplyr) # manipulación de datos
library(tidyr) # limpiar datos
library(outliers) # para datos atípicos

# Datos
Procedemos a importar los datos para cada variable (pestaña Download) de:

- Suicide death rates dataset: https://ourworldindata.org/grapher/suicide-death-rates
- Mental health disorders dataset: https://ourworldindata.org/grapher/people-with-mental-health-disorders
- Population dataset: https://ourworldindata.org/grapher/population

Las comentamos brevemente:
- **Suicide death rates dataset** tiene 6468 registros con 4 variables:`country name`,`country code`,`year` y `suicide death rates`. La columna de `suicide death rates` representa el número de muertes por suicidio por cada 100000 habitantes. Hay datos desde 1990 a 2017.

- **Mental health disorders dataset** tiene 6156 registros con 4 variables:`country name`,`country code`,`year` y `mental disorder count`. Esta última variable contiene el número de personas con problemas y desórdenes mentales (sin incluir problemas de drogas o alcohol) de 1990 a 2016.

- **Population dataset** contiene 46883 registros con 4 variables: `country name`, `country code`, `year` y `population count` desde 1800 a 2019.

In [ ]:
# Importamos y vemos los primeros datos de suicide_death_rates dataset
suicide_death_rates <-read.csv("_data/suicide-death-rates.csv",header = TRUE, 
                    stringsAsFactors = FALSE,
                    col.names = c("country_name","country_code","year","suicide_rate"))
head(suicide_death_rates)

In [ ]:
# Importamos y vemos los primeros datos de mental_health_disorders dataset
mental_health_disorders <-read.csv("_data/people-with-mental-health-disorders.csv", header = TRUE,
                                   stringsAsFactors = FALSE,
                    col.names = c("country_name","country_code","year","mental_disorder_count") )
head(mental_health_disorders)

In [ ]:
# Importamos y vemos los primeros datos de mental_health_disorders dataset
population <-read.csv("_data/population.csv",  header = TRUE,
                      stringsAsFactors = FALSE,
                   col.names = c("country_name","country_code","year","population_count") )
head(population)

Procedemos a unir los conjuntos de datos **Suicide death rates** y **Mental health disorders**.
En base a las variables `country_name`, `country_code` y `year` unimos los 2 conjuntos de datos en un dataframe llamado **joined_rates_data**. Posteriormente, añadimos (a la izquierda) los datos de población basados en las variables comunes `country_name`, `country_code` y `year` como antes.
El dataframe final **joined_data** contiene las variables 
- `country_name`: nombre del país
- `country_code`: acrónimo del nombre del país
- `year`: año del dato
- `suicide_rate` : muertes por suicidio por cada 100000 habitantes 
- `mental_disorder_count`: número de personas con desórdenes mentales 
- `population_count`: número de habitantes de un país un año determinado

In [ ]:
# Unimos (por la derecha) los datos de suicide_death_rates y mental_health_disorders
# y los almacenamos en joined_rates_data
joined_data <- right_join(suicide_death_rates, mental_health_disorders,
                                by = c('country_name','country_code','year'))
head(joined_data)

In [ ]:
# Ahora añadimos a joined_rates_data los datos de population (por la izquierda) 
# para que queden a la derecha (última columna)
joined_data <- left_join(joined_data, population,
                               by= c('country_name','country_code','year'))
head(joined_data)

In [ ]:
# Estructura de los datos
str(joined_data)

# Datos nulos (NA)
Una vez obtenido el conjunto de datos, comprobamos la existencia de datos nulos. Posteriormente, si no es un número muy elevado, procederemos a eliminarlos.

In [ ]:
# Análisis de datos. Buscando datos nulos NA's
table(is.na(joined_data))

Vemos que hay 36126 datos válidos y 810 datos nulos lo que supone un 2.2% de datos nulos respecto al total de datos.

In [ ]:
# porcentaje de datos nulos respecto al total
 810/(36126 + 810)*100

In [ ]:
# Vemos en qué variable hay datos nulos. 
sapply(joined_data, function(x) sum(is.na(x)))

Acabamos de comprobar que aproximadamente un 2% de los datos son NA todos en la variable `population_count`. Una opción es asignar a los valores NA la media o la mediana de los datos de la variable (columna) correspondiente pero, en este caso, no tendría sentido al tratarse de la población por países. Procedemos a eliminarlos con la función `drop_na` almacenando el resultado obtenido en **joined_data_nona**

In [ ]:
# Porcentaje bajo. Eliminamos las filas con algun NA con la función drop_na 
# del paquete tidyr. Almacenamos en joined_data_nona

joined_data_nona <- drop_na(joined_data)

In [ ]:
# Análisis de datos. Buscando datos nulos NA's
table(is.na(joined_data_nona))

In [ ]:
# Resumen de las variables
summary(joined_data_nona)

# Tipos de variables
Con la función str (structure) veamos qué tipo de variables tenemos.

In [ ]:
# Estructura de los datos
str(joined_data_nona)

- las variables `country_name`y `country_code`son caracteres. Estas variables se podrían conventir en factores ya que hay número determinado (195) de países posibles mediante la función as.factor().
- La variable `year` se puede dejar como número entero (sin decimales) ya que no precisamos de fechas concretas.
- - La variable `suicide_rate` se puede dejar como número con decimales.
- Las variables `mental_disorder_count` y `population_count` aparecen como num (en principio con decimales) aunque son números enteros.

Vamos a añadir texto a los resultados con el comando `cat()`.

In [ ]:
# Vemos el tipo de variable
cat("La variable country_name es de tipo:")
class(joined_data_nona$country_name)
cat("La variable country_code es de tipo:")
class(joined_data_nona$country_code)
cat("La variable year es de tipo:")
class(joined_data_nona$year)
cat("La variable suicide_rate es de tipo:")
class(joined_data_nona$suicide_rate)
cat("La variable mental_disorder_count es de tipo:")
class(joined_data_nona$mental_disorder_count)
cat("La variable population_count es de tipo:")
class(joined_data_nona$population_count)

# Preprocesado de datos 1
Una vez eliminados los valores nulos, en este momento nuestros datos cumplen que:
- Cada variable está en una columna
- Cada registro está en una única fila
- Todos los valores están en una única celda

Continuamos eliminando la variable `country_code`al ser redundante y darnos la misma información que la variable `country_name`que conservamos.

In [ ]:
# Eliminamos la columna county_code
joined_data_nona <- subset(joined_data_nona, 
                     select= c('country_name','year',
                               'suicide_rate','mental_disorder_count',
                               'population_count'))
head(joined_data_nona)

# Preprocesado de datos 2
Para calcular la proporción de personas con desórdenes mentales por cada 100000 habitantes, dividimos el número (mental_disorder_count) entre la población (population_count) y lo multiplicamos por 100000. Lo almacenamos en la variable `mental_health_disorders_rate` usando la función `mutate`.

In [ ]:
# Incluimos la variable mental_health_disorders_rate
joined_data_nona <- mutate(joined_data_nona,
                            mental_health_disorders_rate =((mental_disorder_count/population_count)*100000))

head(joined_data_nona)

In [ ]:
summary(joined_data_nona)

# Análisis de datos 1. Valores atípicos (outliers)
Para localizar datos atípicos (outliers), tomamos de nuestro dataframe **joined_rates_data** las variables numéricas `suicide_rate`, `mental_disorder_count`, `population_count`y `mental_health_disorders_rate` y lo llamamos **joined_data_sub**.
Usamos este nuevo dataframe para localizar valores atípicos por el método Z-score (normalizando datos).
Veremos que hay datos atípicos que suponen aproximadamente un 5% del total de datos. Recalcularemos esos datos atípicos por el valor más cercano mediante técnica de capping (sustituirlo par un valor menor del percentil 99 o mayor del percentil 1).

In [ ]:
# Extraemos las variables numéricas de joined_rates_data
# Las almacenamos como joined_data_sub
joined_data_sub <- joined_data_nona %>% dplyr:: select(suicide_rate,mental_disorder_count,population_count,
                                mental_health_disorders_rate) 

summary(joined_data_sub) 

In [ ]:
# Visualización de datos: diagramas de cajas y bigotes de las variables
# la distribución de datos con un boxplot para
# localizar dónde hay valores atípicos
par(mfrow=c(1,2))
boxplot(x = joined_data_sub$suicide_rate, main="suicide_rate")
boxplot(x = joined_data_sub$mental_health_disorders_rate, main="mental_health_disorders_rate")

Confirmamos que hay valores atípicos en las 2 variables `suicide_rate` y `mental_health_disorders_rate`. Utilizamos el método z-score (en comparación con la distribución Z, valores que se alejan de la media más de 3 desviaciones típicas) para ver cuántos hay. 

In [ ]:
# Método z-score para detectar valores atípicos
z.scores <- joined_data_sub  %>% scores(type = "z")
length(which( abs(z.scores) >3 ))

Una vez localizados los valores atípicos (outliers) en las variables numéricas mediante Z-score procedemos a sustituirlos por el valor más cercano mediante **técnica de capping** (sustituirlo por un valor menor del percentil 99 o mayor del percentil 1).

In [ ]:
# Definimos la función cap para aplicar la técnica de capping
cap <- function(x){
quantiles <- quantile( x, c(.05, 0.25, 0.75, .95 ))
x[ x < quantiles[2] - 1.5*IQR(x) ] <- quantiles[1]
x[ x > quantiles[3] + 1.5*IQR(x) ] <- quantiles[4]
x
}

# Aplicamos la función cap a nuestros datos numéricos de joined_data_sub
# Lo almacenamos en joined_data_capped
joined_data_capped <-  as.data.frame(sapply(joined_data_sub, FUN = cap ))
summary(joined_data_capped)

In [ ]:
# Volvemos a representar las variables 
par(mfrow=c(1,2))
boxplot(x = joined_data_capped$suicide_rate, main="suicide_rate")
boxplot(x = joined_data_capped$mental_health_disorders_rate, main="mental_health_disorders_rate")

In [ ]:
# Comprobamos que ya no hay valores atípicos
z.scores <- joined_data_capped  %>% scores(type = "z")
length(which( abs(z.scores) >3 ))

En resumen, en esta práctica hemos unido varias bases de datos, hemos encontrado y eliminado valores nulos y localizado valores atípicos que hemos sustituido mediante la técnica de capping.